In [1]:
import pandas as pd
import numpy as np
import plotly as pl
import plotly.graph_objects as go

In [2]:
# Copy the existing 'plotly_dark' theme
custom_dark_theme = go.layout.Template(pl.io.templates['plotly_dark'])

# Your custom settings
custom_bg_color = 'rgb(15, 9, 28)'  # Your custom background color
custom_font = 'Poppins'  # Your custom font
custom_colorway = ['rgb(255, 185, 54)', '#FF6F59', '#4ECDC4', '#C7F464', '#F7FFF7']

# Update the theme layout
custom_dark_theme.layout.update(
    plot_bgcolor=custom_bg_color,
    paper_bgcolor=custom_bg_color,
    font=dict(family=custom_font, color='white'),
    colorway=custom_colorway,
    margin=dict(t=0, l=0, b=0, r=0),
)

# Add the custom theme to Plotly's templates
pl.io.templates['my_custom_dark_theme'] = custom_dark_theme

# Set the custom theme as default
pl.io.templates.default = 'my_custom_dark_theme'

In [3]:
data_dir = 'processed_data/'

In [4]:
baby_names = pd.read_csv(data_dir + 'baby_name_df.csv')
movies = pd.read_csv(data_dir + 'movie_df.csv', index_col='wiki_ID')

In [5]:
tmp = baby_names[baby_names.name == 'Mia'].pivot(index='year', columns='name', values='percentage')
Mia_data = pd.Series(0.0, index=baby_names.year.unique())
Mia_data[tmp.index] = tmp.Mia
Mia_data.sort_index(inplace=True)

In [6]:
Mia_data.name = 'percentage'
Mia_data.index.name = 'year'

In [7]:
Pulp_fiction_data = movies.loc[54173]
x = Pulp_fiction_data.year
y = Mia_data.loc[x]
name = Pulp_fiction_data['mov_name']

In [13]:
fig = go.Figure()

fig.update_layout(showlegend=False, xaxis_title='Year', yaxis_title='Percentage of given names')
fig.add_trace(go.Scatter(x=Mia_data.index, y=Mia_data, line_shape='spline', name='Mia'))
fig.update_xaxes(range=[baby_names.year.min(), baby_names.year.max()])

fig.add_annotation(
    x=x, y=y, text=name,
    xref='x', yref='y', ax=0, ay=-80,
    font={'size': 16}, align='center',
    showarrow=True, arrowwidth=2,
    borderpad=5, bgcolor='#ff2d42', opacity=0.8
)

fig.add_trace(go.Scatter(
    x=[x], y=[y], text=[name], name='',
    mode='markers', marker={
        'color': '#ff2d42', 'size': 15, 'line': {'width': 5, 'color': custom_bg_color}
    }
))

fig.write_html('influence_curveOnly.html', include_plotlyjs="../../js/plotly-2.12.1.min.js", config={'displayModeBar': False})
fig.show()

In [9]:
range_before = np.arange(x-10, x, 1)
range_after = np.arange(x, x+5, 1)
fit_before = np.poly1d(np.polyfit(range_before, Mia_data[range_before], 1))
fit_after = np.poly1d(np.polyfit(range_after, Mia_data[range_after], 1))

In [14]:
fig.update_traces(selector=0, mode='lines+markers')
fig.update_xaxes(range=[x-12, x+6])
fig.update_yaxes(range=[-0.003, 0.11])
fig.add_vrect(x0=x-10.5, x1=x-0.5, line_width=0, fillcolor=custom_colorway[1], opacity=0.2)
fig.add_vrect(x0=x-0.5, x1=x+4.5, line_width=0, fillcolor=custom_colorway[2], opacity=0.2)

fig.add_trace(go.Scatter(x=Mia_data.index, y=fit_before(Mia_data.index), mode='lines', line={'color': custom_colorway[1]}, text=f'β1 = {fit_before[1]}', name='before'))
fig.add_trace(go.Scatter(x=Mia_data.index, y=fit_after(Mia_data.index), mode='lines', line={'color': custom_colorway[2]}, text=f'β1 = {fit_after[1]}', name='after'))

fig.write_html('influence_regression.html', include_plotlyjs="../../js/plotly-2.12.1.min.js", config={'displayModeBar': False})
fig.show()